Michael O'Donnell - 02/08/2024

STC 510 - Module 5 Essentials

Support Vector Machine (SVM)

For the essentials assignment, this project builds off of the basics of the basics of the assignment. I import and clean the data the same. I borrowed that code from myself for the sake of simplicity. I used this stack overflow question:

https://stackoverflow.com/questions/12693438/alternatives-to-naive-bayes-algorithm

to get ideas for 2 other algorithms/methods to classify text in python. In this page, one author suggests to use Logistic Regression and Support Vector Machine (SVM). This portion of the project follows this article:

https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

on SVM. Me using this builds off my code from the other assignment under a line break.

In [2]:
#import statements
import json
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict 
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

In [3]:
#setting the stop words
english_stopwords = set(stopwords.words('english') + list(punctuation) + ['..','...','nbsp','n\'t'])

In [4]:
#iniatalizing the lemmatizer
lemmatizer = WordNetLemmatizer()

In [5]:
#bringing in the jeopardy JSON file
with open('jeopardy.json') as jeopardy_file:
    jeopardy_contents = jeopardy_file.read()

In [6]:
#parsing the jeopardy JSON file so it can read by python and put into a dataframe
parsed_jeopardy = json.loads(jeopardy_contents)

In [7]:
#puts the JSON questions into a dataframe
df = pd.DataFrame(parsed_jeopardy)

In [8]:
#Cleans the value column and sets the values to integers
df['value'] = df['value'].str.replace('$','')
df['value'] = df['value'].str.replace(',','')
df['value'] = df['value'].str.strip()
df['value'] = pd.to_numeric(df['value'])

In [9]:
#Adds a new column that uses the explained above logic that gives easy questions a 1 and hard questions a 0
df['difficulty'] = np.where(((df['round']=='Jeopardy!') & (df['value'] <= 800)) | ((df['round']=='Double Jeopardy!') & (df['value'] <= 1200)),1,0)

In [10]:
#This is a function that takes each cell given to it from a dataframe that then goes and makes it lowercase,
# tokenizes it, lemmantizes it and puts it back into the original string of the cell 
def manipulate_text(text):
    text = text.lower()
    toke_text = word_tokenize(text)
    wordlist = []
    for word in toke_text:
        if word not in english_stopwords:
            wordlist.append(word)
    wordlist2 = []
    for eachword in wordlist:
        wordlist2.append(lemmatizer.lemmatize(eachword))
    text = [' '.join(wordlist2)]
    text = str(text)
    return text

In [11]:
#This puts the column of question that I want to manipulate for training in the above function
df['question'] = df['question'].apply(manipulate_text)

------------------------------------------

https://ai.stackexchange.com/questions/7202/why-does-training-an-svm-take-so-long-how-can-i-speed-it-up

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html

I used the above sources to help come with to this conclusion. The svm sorting algorithm was taking forever to run on my pc compared to Naive Bayes and Logisitic Regression, so I was afraid I was going to fry my CPU. 

I found that to get around SVMs long processing times, I can just use a small sample of my data. 

So I took 10% of the original dataframe randomly and set it to a new dataframe to use as our data. This ended up helping tremendously. 

For both SVM and Logisitc Regression, I got lucky that I did not have to reprocess my original data in any way.

In [22]:
#Making a smaller dataframe of the original dataframe randomly
df2 = df.sample(frac=.10, replace=True, random_state=1)

----------------------------

In [23]:
#Splits the dataset into training and test data with a 75/25 ratio. X is the questions and Y is the difficulty
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df2['question'], df2['difficulty'], test_size=.25, random_state=1000)

In [24]:
#This is done to transform Categorical data of string type in the data set into numerical values which the model can understand
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [25]:
#Uses Tfidf to vectorize the question data
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df2['question'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [26]:
#fits the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf, Train_Y)

SVC(gamma='auto', kernel='linear')

In [27]:
#predicts the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [28]:
#Prints predictions
print(accuracy_score(predictions_SVM, Test_Y))

0.8030973451327433


The 80% classifcation rate was thrilling for me to see. I was happy to see that all 3 of the different algorithms gave similar results. All 3 testify to the way I wanted the data to be classified as easy and hard. It also shows how even though 3 different approaches were used, all gave similar sorting results in the end which was similar to see. 